In [1]:
import ir_datasets
import pyterrier as pt
print(pt.__version__)

pt.init()
from pathlib import Path
import re
import pandas as pd
from pyterrier.measures import RR, nDCG, MAP
from tqdm.notebook import tqdm

0.13.0


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\Scott\AppData\Local\Temp\ipykernel_11500\3500952728.py:5: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [2]:
antique = ir_datasets.load("antique/train")
print(antique.docs[0])

GenericDoc(doc_id='2020338_0', text="A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy.")


In [3]:
# OPTIONAL - Index the data

idx_path = Path.cwd() / "indices" / "antique_train"

def antique_gen(limit=500000):
    lastdoc = 0
    for elem in tqdm(antique.docs, desc="Generating Antique docs", total=limit):
        if lastdoc >= limit:
            break
        yield {
            "docno": elem.doc_id,
            "text": elem.text,
        }
        lastdoc += 1

max(len(text.encode("utf-8")) for _, text in antique.docs)

if not idx_path.exists() or not any(idx_path.iterdir()):
    indexer = pt.IterDictIndexer(
        str(idx_path),
        meta={
            "docno": 20,
            "text": 4096,
        },
        stemmer="porter",
        stopwords="terrier",
    )

    index_ref = indexer.index(antique_gen())
else:
    print("Indices already exist, skipping creation")

Generating Antique docs:   0%|          | 0/500000 [00:00<?, ?it/s]

14:09:18.074 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (730691_1) - further warnings are suppressed
14:10:33.362 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 2224 empty documents


In [4]:
# Define index paths
index_dir_antique = Path.cwd() / "indices" / "antique_train"
# Load the indexes
#index_msmarco = pt.IndexFactory.of(str(index_dir_msmarco))
index_antique = pt.IndexFactory.of(str(index_dir_antique))

# Use BM25 as the baseline retriever
#retriever_msmarco = pt.BatchRetrieve(index_msmarco, wmodel="BM25")
retriever_antique = pt.BatchRetrieve(index_antique, wmodel="BM25")

C:\Users\Scott\AppData\Local\Temp\ipykernel_11500\2060639876.py:9: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  retriever_antique = pt.BatchRetrieve(index_antique, wmodel="BM25")


In [5]:
# Convert qrels to DataFrames
#qrels_msmarco = pd.DataFrame(msmarco.qrels_iter())
qrels_antique = pd.DataFrame(antique.qrels_iter())

# Convert queries to DataFrames
#queries_msmarco = pd.DataFrame(msmarco.queries_iter())
queries_antique = pd.DataFrame(antique.queries_iter())

# Rename columns for PyTerrier compatibility
#qrels_msmarco.rename(columns={"query_id": "qid", "doc_id": "docno", "relevance": "label"}, inplace=True)
qrels_antique.rename(columns={"query_id": "qid", "doc_id": "docno", "relevance": "label"}, inplace=True)

#queries_msmarco.rename(columns={"query_id": "qid", "text": "query"}, inplace=True)
queries_antique.rename(columns={"query_id": "qid", "text": "query"}, inplace=True)

[INFO] Please confirm you agree to the authors' data usage agreement found at <https://ciir.cs.umass.edu/downloads/Antique/readme.txt>
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel: [00:00] [626kB] [1.65MB/s]
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train-queries.txt   
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train-queries.txt: [00:00] [137kB] [620kB/s]


In [6]:
def clean_query(query):
    query = query.encode("ascii", "ignore").decode()
    query = query.replace("'", "").replace('"', "").replace("`", "")
    query = re.sub(r"\s+", " ", query).strip()
    return query

#queries_msmarco["query"] = queries_msmarco["query"].apply(clean_query)
queries_antique["query"] = queries_antique["query"].apply(clean_query)

In [7]:
#Model imports
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the T5 model
MODEL_ID = "prhegde/t5-query-reformulation-RL"
tokenizer = T5Tokenizer.from_pretrained(MODEL_ID)
model = T5ForConditionalGeneration.from_pretrained(MODEL_ID)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [15]:
# Function to rewrite queries
def rewrite_query(query):
    input_ids = tokenizer(query, return_tensors="pt").input_ids
    with torch.no_grad():
        output = model.generate(input_ids, max_length=35, num_beams=1, do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def clean_query(query):
    query = query.replace("\n", " ").replace("\r", " ").replace("?", "")
    query = query.encode("ascii", "ignore").decode()
    query = re.sub(r"[\"`]", "", query)
    query = query.replace("'", "").replace('"', "").replace("`", "")
    query = re.sub(r"\s+", " ", query).strip()
    query = re.sub(r"[^\w\s]", "", query)
    return query

queries_unchanged = queries_antique[["qid", "query"]]
queries_unchanged["query"] = queries_unchanged["query"].apply(clean_query)
#queries_antique["rewritten_query"] = queries_antique["query"].apply(rewrite_query).apply(clean_query)

#Apply with progress bar
queries_antique["rewritten_query"] = [
    clean_query(rewrite_query(query))
    for query in tqdm(queries_antique["query"], desc="rewriting queries")
]


C:\Users\Scott\AppData\Local\Temp\ipykernel_11500\1477539906.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queries_unchanged["query"] = queries_unchanged["query"].apply(clean_query)


rewriting queries:   0%|          | 0/2426 [00:00<?, ?it/s]

In [16]:
bm25_antique = pt.terrier.Retriever(index_antique, wmodel="BM25")


In [18]:
print(queries_unchanged['query'].head(1))
#print(queries_antique['rewritten_query'].head(1))


print("Unchanged queries")


results_unchanged = pt.Experiment(
    [bm25_antique],
    queries_unchanged,  # Use rewritten queries
    qrels_antique,
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)
queries_antique['query'] = queries_antique['rewritten_query']
print(queries_antique['query'].head(1))

print("Rewritten queries")

results_rewritten = pt.Experiment(
    [bm25_antique],
    queries_antique[["qid", "query"]],  # Use rewritten queries
    qrels_antique,
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)

# Optionally, print the results of both experiments
print("Results for unchanged queries:", results_unchanged)
print("Results for rewritten queries:", results_rewritten)

0    bicep muscle weakness and pain in knees causes
Name: query, dtype: object
Unchanged queries


C:\Users\Scott\AppData\Local\Programs\Python\Python39\lib\site-packages\pyterrier\terrier\retriever.py:283: UserWarning: Skipping empty query for qid 2041828
  warn(


0    bicep muscles causes knee pain
Name: query, dtype: object
Rewritten queries
Results for unchanged queries:                 name     RR@10   nDCG@20        AP
0  TerrierRetr(BM25)  0.243063  0.121909  0.072937
Results for rewritten queries:                 name     RR@10   nDCG@20        AP
0  TerrierRetr(BM25)  0.154593  0.080768  0.047341
